In [11]:
import pandas as pd
import numpy as np
import math
import os
from tqdm.notebook import tqdm
from PyRadioLoc.Utils.GeoUtils import GeoUtils
from PyRadioLoc.Pathloss.Models import Cost231HataModel
from PyRadioLoc.Pathloss.Models import OkumuraHataModel

In [12]:
data = pd.read_csv(DATA_DIR + "/LocTreino_Equipe_4.csv")

# Gera grids 20x20, 30x30 e 40x40

In [13]:
def generate_grids():
    LAT_INI  = min(data['lat'])
    LAT_FIN  = max(data['lat'])
    LONG_INI = min(data['lon'])
    LONG_FIN = max(data['lon'])
    delta_x = max(GeoUtils.distanceInKm(LAT_INI, LONG_INI, LAT_INI, LONG_FIN),
                  GeoUtils.distanceInKm(LAT_FIN, LONG_INI, LAT_FIN, LONG_FIN))
    delta_y = max(GeoUtils.distanceInKm(LAT_INI, LONG_INI, LAT_FIN, LONG_INI),
                  GeoUtils.distanceInKm(LAT_INI, LONG_FIN, LAT_FIN, LONG_FIN))
    R = [20, 30, 40]               # dimensoes de cada célula m x m

    # Para cada um das dimensoes de R calcula uma matriz de fingerprint e salva e um csv 
    for num in R:    
        n_cells_x = math.ceil((delta_x * 1000) / num)
        n_cells_y = math.ceil((delta_y * 1000) / num)

        inc_lat = (LAT_FIN - LAT_INI) / n_cells_x
        inc_lng = (LONG_FIN - LONG_INI) / n_cells_y

        result = []

        for i in range(n_cells_x):
            for j in range(n_cells_y):
                lat = LAT_INI + i * inc_lat
                lng = LONG_INI + j * inc_lng
                result.append((lat, lng))

        # salva para o CSV
        fingerprint_map = pd.DataFrame(result, columns=['lat', 'lon'])
        fingerprint_map.to_csv("./grid/grid_{}.csv".format(num), index=False)

        print(n_cells_x, n_cells_y)

# Aplica o modelo teorico de Cost231HataModel e gera os fingerprints

In [15]:
FREQ_ERBS = 1800
MODEL = Cost231HataModel(FREQ_ERBS)

In [18]:
def calculate_erb_rssi(fingerprint, erb, model):
    """
    Calcula todos os valores de rssi de uma erb para todos os pontos de um fingerprint
    """
    rssi_list = []
    eirp = erb['Eirp']
    lat_erb = erb['lat']
    lon_erb = erb['lon']
    rssiId = erb['RssiId']

    for n, point in fingerprint.iterrows():
        lat = point['lat']
        lon = point['lon']
        
        distance = GeoUtils.distanceInKm(lat_erb, lon_erb, lat, lon)
        pathloss = model.pathloss(distance)
        rssi = eirp - pathloss
        rssi_list.append(rssi)
    
    return rssi_list

def generate_grid(erbs, fingerprint):
    """
    Gera um fingerprint e retorna como dataFrame
    """
    rssi_dict = {}
    for idx, erb in tqdm(erbs.iterrows()):
        rssiId = erb['RssiId']
        rssi_dict[rssiId] = calculate_erb_rssi(fingerprint, erb, MODEL)
    grid = pd.DataFrame(rssi_dict)
    return grid
    
def fingerprints_to_csv():
    """
    Gera todos os fingerprints e salva em csv
    """
    dims = [20, 30, 40]
    erbs = pd.read_csv("./database/Bts.csv")
    for dim in tqdm(dims):
        fingerprint = pd.read_csv("./grid/grid_{}.csv".format(dim))
        grid = generate_grid(erbs, fingerprint)
        grid.insert(0, 'lat', fingerprint['lat'], True)
        grid.insert(1, 'lon', fingerprint['lon'], True)
        grid.to_csv("./fingerprint/fingerprint_{}.csv".format(dim), index=False)

In [19]:
fingerprints_to_csv()